# Organism WRONG WRONG WRONG
In the simplest model, there is only a class for the **Organism**. Each organism reproduces asexually with the parameter that an individual's population $f(t)$ is governed by $f(t) = f(0)e^{st}$ where $s$ is a fitness parameter determined by an organism's individual fitness. Some individuals will acquire a beneficial mutation with probability $U_b$, increasing their fitness, whereas others may acquire a detrimental mutation with probability $U_d$. To implement this mechanistically, *while keeping population size constant*, we first convert the fitness $s$ to an expected number of offspring for each organism.

In [81]:
import numpy as np
import math
import random

class Population:
    def __init__(self, Ub, Ud, size):
        """
        Creates Population (see Organism specs)
        """
        self.Ub = Ub
        self.Ud = Ud
        self.gens = [[]]
        self.size = size
        
        for i in range(self.size):
            self.gens[0].append(Organism(np.random.random(), Ub, Ud, [len(self.gens)-1, i]))
            
    def __str__(self):
        return f'population with size {len(self.gens[-1])}, {len(self.gens)} generations, Ub {self.Ub}, Ud {self.Ud}'
            
    def reproduce(self):
        
        self.gens.append([])
        weights = [math.exp(org.s) for org in self.gens[-2]]
        
        for i in range(self.size):
            choice = random.choices(self.gens[-2], weights=weights)[0]
            self.gens[-1].append(choice.reproduce([len(self.gens)-1, i]))

class Organism:
    def __init__(self, s, Ub, Ud, index, parent=None):
        """
        Creates Organism object with fitness s, beneficial mutation rate Ub, detrimental mutation rate Ud, and
        a parent, which is either another Organism object or None if in the parent generation
        index = [gen, number in gen]
        """
        self.s = s
        self.Ub = Ub
        self.Ud = Ud
        self.index = index
        self.parent = parent
        self.children = []
    
    def __str__(self):
        return f'organism {self.index}'
        
    def reproduce(self, index):
        """
        reproduces and gives mutation accordingly
        adds resultant Organism to self.children
        returns resultant Organism
        """
        
        if np.random.random() < self.Ub:
            self.s += self.s * 0.01
        
        if np.random.random() < self.Ud:
            self.s -= self.s * 0.01
        
        child = Organism(self.s, self.Ub, self.Ud, index, parent=self)
        self.children.append(child)
        
        return child

In [84]:
from pprint import pprint
import time

start_time = time.time()

bacteria = Population(10**(-4), 10**(-3), 10**1)

num_gens = 99
for i in range(num_gens):
    bacteria.reproduce()

print(bacteria)
print(np.asarray(bacteria.gens).shape)
# for gen in bacteria.gens:
#     pprint([str(org) for org in gen])
    
print("--- %s seconds ---" % (time.time() - start_time))

population with size 10, 100 generations, Ub 0.0001, Ud 0.001
(100, 10)
--- 0.013664007186889648 seconds ---


In [ ]:
import networkx as nx
import pydot
from networkx.drawing.nx_pydot import graphviz_layout

G = nx.DiGraph()

ancestor = random.choices(bacteria.gens[0])

G.add_node(ancestor)

# trace lineage of each organism in parent generation
while 


nx.draw(G, with_labels=True, node_color = ['blue'])

print("--- %s seconds ---" % (time.time() - start_time))